In [1]:
import numpy as np
import pandas as pd

In [2]:
import pyedflib # ref: https://pyedflib.readthedocs.io/en/latest/
import pandas as pd
import numpy as np
import os
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import fnmatch
from modules import *
from tqdm import tqdm
import logging

# the format of plot will be THRES/CONSEC/TOTAL_SIZZE
EEG_THRES = 25
EEG_CONSEC = 300
EEG_TOTAL_SIZE = 1500

EEG2_THRES = 20
EEG2_CONSEC = 500
EEG2_TOTAL_SIZE = 1500

EOG_THRES = 120
EOG_CONSEC = 2500
EOG_TOTAL_SIZE = 2500
extend = 30 

def reprval(text):
    thres = eval(f"{text}_THRES")
    consec = eval(f"{text}_CONSEC")
    total_size = eval(f"{text}_TOTAL_SIZE")
    return f"{text}_{thres}-{consec}-{total_size}"

read_folder = f'./clips_front_extend_{extend}/'
write_folder = f'plot_{extend}_{reprval("EEG")}_{reprval("EEG2")}_{reprval("EOG")}/'

# logging.basicConfig(filename=write_folder+'print.log', encoding='utf-8', level=logging.DEBUG)
print(write_folder)

os.makedirs(write_folder, exist_ok=True)
pattern1 = '*-PSG.edf'
pattern2 = '*-Hypnogram.edf'

psg_list = sorted([f for f in os.listdir(read_folder) if fnmatch.fnmatch(f, pattern1)])
hypnogram_list = sorted([f for f in os.listdir(read_folder) if fnmatch.fnmatch(f, pattern2)])
psg_iter = iter(psg_list)
hypnogram_iter = iter(hypnogram_list)

channels = ['EEG Fpz-Cz', 'EEG Pz-Oz', 'EOG horizontal']
channel = channels[1]

for i, (psg_id, hypnogram_id) in enumerate(zip(psg_iter, hypnogram_iter)):
    print(f"{i}/{len(psg_list)}", end="\r")
    signal_path = os.path.join(read_folder, psg_id)
    label_path = os.path.join(read_folder, hypnogram_id)
    edf_signal = pyedflib.EdfReader(signal_path)
    edf_label = pyedflib.EdfReader(label_path)
    annotations = edf_label.readAnnotations()
    start = edf_signal.getStartdatetime()
    signals, frequencies = edf_signal.getSignalLabels(), edf_signal.getSampleFrequencies()
    
    data = []
    for ch_idx, sig_name, freq in zip( range(len(signals)), signals, frequencies,):
        sig = edf_signal.readSignal(chn=ch_idx)
        idx = pd.date_range(  start=start, periods=len(sig), freq=pd.Timedelta(1 / freq, unit="s") )
        data += [pd.Series(sig, index=idx, name=sig_name)]
    
    # create DataFrames
    annotations_df = pd.DataFrame(annotations)
    annotations_df = annotations_df.T
    annotations_df.rename(columns={0: "Onset", 1: "Duration", 2:"Annotations"}, inplace=True)
    signal_df =pd.concat(data[0:3], axis=1)
    
    break

plot_30_EEG_25-300-1500_EEG2_20-500-1500_EOG_120-2500-2500/


In [9]:
signal_df.index[1] > signal_df.index[0]

True

In [11]:
signal_df.index[1]

Timestamp('1989-04-25 00:33:30.010000')

In [20]:
s = signal_df.index[1] - pd.Timedelta(minutes=1)
e = signal_df.index[1] + pd.Timedelta(minutes=1)

In [27]:
gogo = pd.Series(np.zeros(len(signal_df)), index=signal_df.index)

In [34]:
gogo.loc[s:e] = True

In [32]:
gogo.loc[s:e]

1989-04-25 00:33:30.000    True
1989-04-25 00:33:30.010    True
1989-04-25 00:33:30.020    True
1989-04-25 00:33:30.030    True
1989-04-25 00:33:30.040    True
                           ... 
1989-04-25 00:34:29.970    True
1989-04-25 00:34:29.980    True
1989-04-25 00:34:29.990    True
1989-04-25 00:34:30.000    True
1989-04-25 00:34:30.010    True
Freq: 10L, Length: 6002, dtype: object